In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import psycopg2
import plotly.express as px

# Fast to .csv
def csv(df, name):
    df.to_csv(f'{name}.csv',index=False)

# Fast to .xlsx
def excel(df, name):
    df.to_excel(f'{name}.xlsx',index=False)
    
def authorization():
    login_name = ''
    login_password = ''
    IP = ''
    engine_str = f'mssql+pymssql://{login_name}:{login_password}@{IP}:1433/monitoring'
    engine = create_engine(engine_str)
    return engine

In [2]:
def execute_data_bd():
    query =f'''SELECT
	YEAR(GMT) AS year,
	MONTH(GMT) AS month,
	DATEPART(HOUR, DATEADD(HOUR, 8, GMT)) AS hour,
	HEADING,
	lat,
	lon,
	AVG(SPEED) AS average_speed
FROM
	monitoring.dbo.transport
WHERE
	GMT > '2022-01-01T00:00:00'	AND GMT < '2022-02-01T00:00:00'
	AND DATEPART(HOUR, DATEADD(HOUR, 8, GMT)) >= 5
	AND SPEED >= 0
	AND SPEED <= 61
GROUP BY
	YEAR(GMT),
	MONTH(GMT),
	DATEPART(HOUR, DATEADD(HOUR, 8, GMT)),
	HEADING,
	lat,
	lon
ORDER BY 
	YEAR(GMT),
	MONTH(gmt),
	DATEPART(HOUR, DATEADD(HOUR, 8, GMT))

    '''
    filtered_lat_lon = pd.read_sql_query(query,authorization())
    return filtered_lat_lon

In [3]:
%%time
df = execute_data_bd()

CPU times: user 36.5 s, sys: 3.81 s, total: 40.3 s
Wall time: 8min 37s


In [5]:
df.month.unique()

array([1])

In [9]:
df

,year,month,hour,HEADING,lat,lon,average_speed
0,2022,1,5,0.0,52.256699,104.364227,15.0
1,2022,1,5,0.0,52.270916,104.325447,0.0
2,2022,1,5,0.0,52.271355,104.326111,0.0
3,2022,1,5,0.0,52.271431,104.327698,0.0
4,2022,1,5,0.0,52.271519,104.328651,0.0
...,...,...,...,...,...,...,...
3084490,2022,1,23,359.0,52.272530,104.327370,0.0
3084491,2022,1,23,359.0,52.276043,104.257591,17.0
3084492,2022,1,23,359.0,52.281219,104.295845,10.0
3084493,2022,1,23,359.0,52.281395,104.295982,14.0


In [6]:
def authorization_pg():
    login_name = ''
    login_password = ''
    IP = ''
    engine_str = f'postgresql://{login_name}:{login_password}@{IP}:5432/postgres'
    engine = create_engine(engine_str)
    return engine

In [17]:
 def generate():
    d = {
        'year':df.year
        ,'month':df.month
        ,'hour':df.hour
        ,'heading':df.HEADING
        ,'lat':df.lat
        ,'lon':df.lon
        ,'average_speed':df.average_speed
    }
    df_into = pd.DataFrame(data=d)
    return df_into

In [19]:
 def into_pg_table(pg_table_name):
    dataframe = generate()
    connector = authorization_pg()
    dataframe.to_sql(
        name = pg_table_name,
        con = connector,
        chunksize = 10000,
        index = False,
        if_exists = 'append'
    )

In [20]:
%%time
into_pg_table('dbo_average_speed_tramway')

CPU times: user 41.6 s, sys: 336 ms, total: 41.9 s
Wall time: 1min 4s
